In [1]:
%load_ext autoreload
%autoreload 2
import pandas as pd 
import numpy as np
import os 
import glob
from government_functions import * 
import ast 
import warnings
warnings.formatwarning = lambda msg, *args, **kwargs: f'{msg}\n'

In [2]:
file_name='Czech.csv'
df=process_gov_csv(f'government/{file_name}')

 now, onto matching :D 

In [3]:
final_df=pd.read_excel('results/complete_dataframe.xlsx',index_col=0)

In [4]:
#all_elections=final_df.columns.to_numpy()
Czech_df=final_df[[col for col in final_df.columns if col.startswith('Czech')]] #this is case sensitive... 
Czech_elections=Czech_df.columns.to_numpy()

In [5]:
def match_ministries_and_elections(countryname,election_dates,df):
    ## takes in the name of the country, a list of election dates and the df from process_gov_csv
    ## to accomodate errors from the political yearbook dataset, begin and end of election cycles are off-set by one month
    ## see i.e. that Germany 09/2013 elected ministers are appointed 31.08.2013 by the political yearbook data
    ## outputs a dict with (election_date,df_for_electionperiod)
    ## df_for_electionperiod has cols: 'Position' and 'Party'

    sorted_elections= sorted(pd.to_datetime(election_dates, format=f'{countryname}-%b-%y'))
    election_period_dict = {}
    for i in range((len(sorted_elections))): 
        #get timeframe for ministerial appointments per election (start: election -1 month)
        start=sorted_elections[i]
        adjusted_start= start - pd.DateOffset(months=1)
        end = (sorted_elections[i+1] if i+1<len(sorted_elections) else pd.Timestamp.today())- pd.DateOffset(months=1)#.strftime('-%b-%y')
        #select all datapoints in this timeframe
        boolean_mask = ((df['Start']>=adjusted_start)&(df['Start']<end))
        df_for_electionperiod=df[boolean_mask]
        key=start.strftime('-%b-%y')
        #however keep correct election date as key 
        election_period_dict[key]=df_for_electionperiod
        
    return election_period_dict

In [6]:
test=match_ministries_and_elections('Czech',Czech_elections,df)

In [ ]:
test.keys()

In [ ]:
test.get('-Oct-21')

In [7]:
test_2,_=starting_gov_dict(test,'Czech')

Czech-Jun-02: Use government from 2002-07-15 00:00:00


In [ ]:
test_2.get('-Oct-21')

In [ ]:
for date,dataframe in test_2.items():
        if not dataframe.empty:
            party_str=Czech_df.at['parties',f'Czech{date}']
            parties=ast.literal_eval(party_str) # parses string to list  
            #create dicts
            ministry_dict,unweighted_dict,weighted_dict=get_ministry_dicts(dataframe,parties) 
            #translate dicts into lists / arrays corrosponding to the party list     
            ministy_list=[]
            unweighted_array=np.zeros(len(parties))  
            weighted_array=np.zeros(len(parties))  
            for i,(party,value) in enumerate(ministry_dict.items()): 
                ministy_list.append(value)
            for i,(party,value) in enumerate(unweighted_dict.items()):
                unweighted_array[i]=value
            for i,(party,value) in enumerate(weighted_dict.items()):
                weighted_array[i]=value
            
            # write list/arrays to final_df:
            if 'Ministers' not in final_df.index: 
                final_df.loc['Ministers']=[np.nan] * len(final_df.columns)   
            if 'unweighted' not in final_df.index: 
                final_df.loc['unweighted']=[np.nan] * len(final_df.columns)   
            if 'weighted' not in final_df.index: 
                final_df.loc['weighted']=[np.nan] * len(final_df.columns) 
            final_df.at['Ministers',f'Czech{date}']=ministy_list
            final_df.at['unweighted',f'Czech{date}']=unweighted_array
            final_df.at['weighted',f'Czech{date}']=weighted_array 

In [ ]:
def get_ministry_dicts(df,parties): 
    ## takes in a df with cols position and party and a list of all parties like a value from governments_dict
    ## creates 3 dicts, which store the following information
    
    ## uses fuzzy matching to catch cases where the party of the minister is written differently from the party name in the parliament 
    dict_1={} #(party,list_of_ministries)
    dict_2={} #(party,number_of_ministries)
    dict_3={} #(party,weighted_number_of_ministries) ## weighting prime minister = 3, all other 1
    prime_minister=df.iloc[0]['Position'] #prime minister, chancellor whatever is always the first entry of the df, if created correctly from political yearbook data
    
    for party in parties:
        #check for perfect matches
        ministries=df[df['Party']==party]['Position'].tolist()
        if not ministries: #if no match for this party
            reverse_matches_dict = {}
            for partyy in df['Party'].unique(): 
               best_fitting_party,score=process.extractOne(partyy,parties)
               if score>40: 
                reverse_matches_dict[best_fitting_party] =partyy
            ministries=df[df['Party']==reverse_matches_dict.get(party)]['Position'].tolist()
        dict_1[party]=ministries
        dict_2[party]=len(ministries)
        weighted_ministries= len(ministries) + 2 if prime_minister in ministries else len(ministries)
        dict_3[party]=weighted_ministries
    return dict_1,dict_2,dict_3

In [ ]:
party_str=Czech_df.at['parties',f'Czech-Oct-21']
parties=ast.literal_eval(party_str)
#ministry_dict,unweighted_dict,weighted_dict=get_ministry_dicts(test_2.get('-Oct-21'),parties)

In [ ]:
df=df[df['Start']>pd.to_datetime('Oct-21', format='%b-%y')]
for party in parties:
        ministries=df[df['Party']==party]['Position'].tolist()
        #print(party,ministries)
        if not ministries: #if no match for this party
            reverse_matches_dict = {}
            for partyy in df['Party'].unique(): 
               best_fitting_party,score=process.extractOne(partyy,parties)
               if score > 40: 
                  reverse_matches_dict[best_fitting_party] =partyy
            ministries=df[df['Party']==reverse_matches_dict.get(party)]['Position'].tolist()
            print(f'matched {party} with {reverse_matches_dict.get(party)} to get {ministries}')
                

In [ ]:
from fuzzywuzzy import fuzz
from fuzzywuzzy import process

In [ ]:
for partyy in df['Party'].unique(): 
    best_fitting_party,score=process.extractOne(partyy,parties)
    reverse_matches_dict[best_fitting_party] = partyy

In [ ]:
def starting_gov_dict(election_period_dict,countryname): 
    ## takes in a dict from match_ministries_and_elections and the countryname for the warning function
    ## outputs a dict which matches {election_date,(position,party)}
    ## assumes that the start date which appears most often in the period after the election is the date of inauguration of the new cabinet 
    ## includes some warnings for "suspicious results"
    governments_dict = {}
    edge_cases = []
    for election, df_for_election_period in election_period_dict.items():
        mode = df_for_election_period['Start'].mode() #check which is the most often appearing start date for this period
        if not mode.empty:
            start_date = mode[0] #set this date as the beginning for the new government
            starting_gov = df_for_election_period.loc[df_for_election_period['Start'] == start_date, ['Position', 'Party']]
            
            # Handling of cases where the found government was very late in the election period --> likely errors
            election_date = pd.to_datetime(election, format='-%b-%y')
            if not (election_date.year <= start_date.year <= election_date.year + 1):
                shorter_df = df_for_election_period[(df_for_election_period['Start'] > election_date-pd.DateOffset(months=1)) & (df_for_election_period['Start'] <= (election_date + pd.DateOffset(years=1)))]
                new_mode=shorter_df['Start'].mode()
                if not new_mode.empty:
                    start_date = new_mode[0] #set this new date as the beginning for the new government
                    starting_gov = shorter_df.loc[shorter_df['Start'] == start_date, ['Position', 'Party']]
                    #if start_date != mode[0]:  # Check if new mode is different from the original mode
                    warnings.warn(f"{countryname}{election}: Use government from {new_mode[0]}")
                else:
                    warnings.warn(f"{countryname}{election}: No suitable government found within a year after the election. Fall back to late government at {mode[0]}.")
                    edge_cases.append(f"{countryname}{election}")
            # Warning if the found government has few ministers
            if starting_gov.shape[0] <= 8:
                warnings.warn(f"{countryname}-{election}: Only found {starting_gov.shape[0]} ministers.")

            
            
            # Warning if the found government was before the actual election 
            if start_date < election_date:
                warnings.warn(f"{countryname}-{election}: Careful, check db error. Found Government at {start_date}")

        else:
            #warnings.warn(f"No mode found for {countryname}-{election}")
            starting_gov = pd.DataFrame(columns=['Position', 'Party']) #add empty df when no government info is given for the period
        
        governments_dict[election] = starting_gov
    return governments_dict,edge_cases


create loop for all countries: 

In [ ]:

special_cases= []
for csv_file in glob.glob(os.path.join('government/','*.csv')): 
    file= os.path.basename(csv_file)
    countryname,_=os.path.splitext(file)
    #create country government dataframe
    df=process_gov_csv(csv_file)
    #grab part of final_df that is relevant for the country
    country_df=final_df[[col for col in final_df.columns if col.startswith(countryname)]] #this is case sensitive... 
    elections=country_df.columns.to_numpy() # list of elections in the country
    #get all election_df's (currently not really necessary but might be if stability or changes in government becomre relevant)
    election_period_dict=match_ministries_and_elections(countryname,elections,df)
    #subset election_df's for only the first ministers in an election period (initial government)
    government_dict,edge_cases=starting_gov_dict(election_period_dict,countryname)
    special_cases.extend(edge_cases)
    ##loop over all election periods: ##
    for date,dataframe in government_dict.items():
        if not dataframe.empty:
            party_str=country_df.at['parties',f'{countryname}{date}']
            parties=ast.literal_eval(party_str) # parses string to list  
            #create dicts
            ministry_dict,unweighted_dict,weighted_dict=get_ministry_dicts(dataframe,parties) 
            #translate dicts into lists / arrays corrosponding to the party list     
            ministy_list=[]
            unweighted_array=np.zeros(len(parties))  
            weighted_array=np.zeros(len(parties))  
            for i,(party,value) in enumerate(ministry_dict.items()): 
                ministy_list.append(value)
            for i,(party,value) in enumerate(unweighted_dict.items()):
                unweighted_array[i]=value
            for i,(party,value) in enumerate(weighted_dict.items()):
                weighted_array[i]=value
            
            # write list/arrays to final_df:
            if 'Ministers' not in final_df.index: 
                final_df.loc['Ministers']=[np.nan] * len(final_df.columns)   
            if 'unweighted' not in final_df.index: 
                final_df.loc['unweighted']=[np.nan] * len(final_df.columns)   
            if 'weighted' not in final_df.index: 
                final_df.loc['weighted']=[np.nan] * len(final_df.columns) 
            final_df.at['Ministers',f'{countryname}{date}']=ministy_list
            final_df.at['unweighted',f'{countryname}{date}']=unweighted_array
            final_df.at['weighted',f'{countryname}{date}']=weighted_array 



    



In [ ]:
with pd.ExcelWriter('combined_dataframe.xlsx', engine='xlsxwriter') as writer:
    final_df.to_excel(writer)